### 1 - Import test data

In [16]:
import pandas as pd
import pandas_ta as ta

df = pd.read_csv("nasdaq.csv")
# df["Datetime"]=df["Datetime"].str.replace("-05:00","")
df['Datetime']=pd.to_datetime(df['Datetime'], format='ISO8601', utc=True)

df=df[df.High!=df.Low]
df.set_index("Datetime", inplace=True)


In [17]:
df["EMA_slow"]=ta.ema(df.Close, length=50)
df["EMA_fast"]=ta.ema(df.Close, length=12)
df['RSI']=ta.rsi(df.Close, length=10)
my_bbands = ta.bbands(df.Close, length=15, std=1.5)
df['ATR']=ta.atr(df.High, df.Low, df.Close, length=7)
df=df.join(my_bbands)
df

Open          High           Low  \
Datetime                                                              
2024-01-16 14:30:00+00:00  14905.665039  14938.710938  14881.009766   
2024-01-16 14:45:00+00:00  14876.389648  14910.701172  14866.360352   
2024-01-16 15:00:00+00:00  14902.736328  14934.326172  14892.898438   
2024-01-16 15:15:00+00:00  14934.433594  14977.805664  14925.548828   
2024-01-16 15:30:00+00:00  14974.114258  14992.057617  14954.064453   
...                                 ...           ...           ...   
2024-03-11 18:45:00+00:00  16057.845703  16076.823242  16053.979492   
2024-03-11 19:00:00+00:00  16057.684570  16058.118164  16030.704102   
2024-03-11 19:15:00+00:00  16030.795898  16034.460938  16012.697266   
2024-03-11 19:30:00+00:00  16017.302734  16034.933594  16007.973633   
2024-03-11 19:45:00+00:00  16032.431641  16040.884766  16003.586914   

                                  Close     Volume  Dividends  Stock Splits  \
Datetime                                                                      
2024-01-16 14:30:00+00:00  14885.206055  242955046        0.0           0.0   
2024-01-16 14:45:00+00:00  14910.701172  280490855        0.0           0.0   
2024-01-16 15:00:00+00:00  14931.666016  221621025        0.0           0.0   
2024-01-16 15:15:00+00:00  14974.386719  244870000        0.0           0.0   
2024-01-16 15:30:00+00:00  14987.386719  265508000        0.0           0.0   
...                                 ...        ...        ...           ...   
2024-03-11 18:45:00+00:00  16059.698242   94816000        0.0           0.0   
2024-03-11 19:00:00+00:00  16030.704102  107438000        0.0           0.0   
2024-03-11 19:15:00+00:00  16017.939453  128685000        0.0           0.0   
2024-03-11 19:30:00+00:00  16033.071289  160824000        0.0           0.0   
2024-03-11 19:45:00+00:00  16021.193359  311782000        0.0           0.0   

                               EMA_slow      EMA_fast        RSI        ATR  \
Datetime                                                                      
2024-01-16 14:30:00+00:00           NaN           NaN        NaN        NaN   
2024-01-16 14:45:00+00:00           NaN           NaN        NaN        NaN   
2024-01-16 15:00:00+00:00           NaN           NaN        NaN        NaN   
2024-01-16 15:15:00+00:00           NaN           NaN        NaN        NaN   
2024-01-16 15:30:00+00:00           NaN           NaN        NaN        NaN   
...                                 ...           ...        ...        ...   
2024-03-11 18:45:00+00:00  16057.626400  16049.013237  51.092717  29.438102   
2024-03-11 19:00:00+00:00  16055.062371  16045.684304  43.394325  29.374679   
2024-03-11 19:15:00+00:00  16051.526855  16040.639785  40.415499  28.287392   
2024-03-11 19:30:00+00:00  16049.769182  16039.263695  45.356249  28.097759   
2024-03-11 19:45:00+00:00  16047.047675  16035.978180  42.297257  29.412058   

                             BBL_15_1.5    BBM_15_1.5    BBU_15_1.5  \
Datetime                                                              
2024-01-16 14:30:00+00:00           NaN           NaN           NaN   
2024-01-16 14:45:00+00:00           NaN           NaN           NaN   
2024-01-16 15:00:00+00:00           NaN           NaN           NaN   
2024-01-16 15:15:00+00:00           NaN           NaN           NaN   
2024-01-16 15:30:00+00:00           NaN           NaN           NaN   
...                                 ...           ...           ...   
2024-03-11 18:45:00+00:00  16021.071082  16044.040495  16067.009907   
2024-03-11 19:00:00+00:00  16024.741283  16045.150977  16065.560670   
2024-03-11 19:15:00+00:00  16022.035729  16044.290430  16066.545130   
2024-03-11 19:30:00+00:00  16021.272409  16042.110091  16062.947774   
2024-03-11 19:45:00+00:00  16018.775612  16041.006966  16063.238321   

                           BBB_15_1.5  BBP_15_1.5  
Datetime                                           
2024-01-16 14:30:00+00:00  

In [18]:
def ema_signal(df, current_candle, backcandles):
    df_slice = df.reset_index().copy()
    # Get the range of candles to consider
    start = max(0, current_candle - backcandles)
    end = current_candle
    relevant_rows = df_slice.iloc[start:end]

    # Check if all EMA_fast values are below EMA_slow values
    if all(relevant_rows["EMA_fast"] < relevant_rows["EMA_slow"]):
        return 1
    elif all(relevant_rows["EMA_fast"] > relevant_rows["EMA_slow"]):
        return 2
    else:
        return 0


df=df[-10000:-1]
from tqdm import tqdm
tqdm.pandas()
df.reset_index(inplace=True)
df['EMASignal'] = df.progress_apply(lambda row: ema_signal(df, row.name, 7) , axis=1) #if row.name >= 20 else 0

100%|██████████| 1013/1013 [00:00<00:00, 3539.56it/s]
/var/folders/b9/tv8kqzz13cq_hsmncf9m9bs00000gp/T/ipykernel_44888/3663350097.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['EMASignal'] = df.progress_apply(lambda row: ema_signal(df, row.name, 7) , axis=1) #if row.name >= 20 else 0


In [19]:
def total_signal(df, current_candle, backcandles):
    if (ema_signal(df, current_candle, backcandles)==2
        and df.Close[current_candle]<=df['BBL_15_1.5'][current_candle]
        #and df.RSI[current_candle]<60
        ):
            return 2
    if (ema_signal(df, current_candle, backcandles)==1
        and df.Close[current_candle]>=df['BBU_15_1.5'][current_candle]
        #and df.RSI[current_candle]>40
        ):

            return 1
    return 0

df['TotalSignal'] = df.progress_apply(lambda row: total_signal(df, row.name, 7), axis=1)

100%|██████████| 1013/1013 [00:00<00:00, 1887.52it/s]
/var/folders/b9/tv8kqzz13cq_hsmncf9m9bs00000gp/T/ipykernel_44888/2955075913.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['TotalSignal'] = df.progress_apply(lambda row: total_signal(df, row.name, 7), axis=1)


In [20]:
df[df.TotalSignal != 0].head(20)

Datetime          Open          High           Low  \
50  2024-01-17 20:30:00+00:00  14797.895508  14830.668945  14796.972656   
51  2024-01-17 20:45:00+00:00  14827.496094  14865.621094  14827.496094   
66  2024-01-18 18:00:00+00:00  14961.536133  14961.746094  14926.851562   
67  2024-01-18 18:15:00+00:00  14930.028320  14932.086914  14911.108398   
119 2024-01-22 18:15:00+00:00  15365.522461  15366.959961  15333.734375   
137 2024-01-23 16:15:00+00:00  15365.431641  15366.166992  15347.495117   
174 2024-01-24 19:00:00+00:00  15562.630859  15566.400391  15536.125000   
175 2024-01-24 19:15:00+00:00  15534.116211  15536.847656  15500.612305   
178 2024-01-24 20:00:00+00:00  15521.972656  15536.948242  15503.805664   
179 2024-01-24 20:15:00+00:00  15504.479492  15504.479492  15479.921875   
212 2024-01-26 15:30:00+00:00  15497.358398  15520.322266  15496.233398   
213 2024-01-26 15:45:00+00:00  15515.293945  15531.131836  15513.611328   
234 2024-01-29 14:30:00+00:00  15470.274414  15490.479492  15450.336914   
235 2024-01-29 14:45:00+00:00  15481.767578  15489.355469  15467.566406   
236 2024-01-29 15:00:00+00:00  15480.929688  15490.365234  15458.584961   
271 2024-01-30 17:15:00+00:00  15555.024414  15555.024414  15511.642578   
300 2024-01-31 18:00:00+00:00  15311.599609  15325.294922  15309.329102   
306 2024-01-31 19:30:00+00:00  15271.086914  15394.140625  15262.717773   
366 2024-02-05 15:00:00+00:00  15590.916016  15594.040039  15550.808594   
367 2024-02-05 15:15:00+00:00  15561.216797  15561.216797  15512.854492   

            Close     Volume  Dividends  Stock Splits      EMA_slow  \
50   14826.054688  158802000        0.0           0.0  14806.527390   
51   14857.411133  314128000        0.0           0.0  14811.373461   
66   14930.969727  121763000        0.0           0.0  14949.278686   
67   14921.500977  108418000        0.0           0.0  14946.633190   
119  15335.146484  110024000        0.0           0.0  15343.389118   
137  15349.096680  142008000        0.0           0.0  15365.824037   
174  15536.125000   89384000        0.0           0.0  15549.982180   
175  15508.082031  111860000        0.0           0.0  15545.991690   
178  15504.426758  111519000        0.0           0.0  15539.180633   
179  15480.376953  131931000        0.0           0.0  15533.580283   
212  15515.424805  160741000        0.0           0.0  15496.994215   
213  15530.250977  144685000        0.0           0.0  15500.161525   
234  15479.786133  190283159        0.0           0.0  15470.843021   
235  15484.051758  230709146        0.0           0.0  15472.100996   
236  15487.655273  232599309        0.0           0.0  15473.582355   
271  15512.651367  102038000        0.0           0.0  15558.784380   
300  15325.051758  101316000        0.0           0.0  15350.017040   
306  15388.052734  181789000        0.0           0.0  15336.534167   
366  15562.251953  252184593        0.0           0.0  15592.484758   
367  15531.022461  225048000        0.0           0.0  15586.631206   

         EMA_fast        RSI        ATR    BBL_15_1.5    BBM_15_1.5  \
50   14802.271546  57.068970  24.280269  14764.212927  14794.156055   
51   14812.296925  64.511322  26.464844  14759.390226  14797.190039   
66   14971.660567  43.997459  31.450915  14954.926245  14996.297461   
67   14962.540642  41.708252  29.954809  14941.208454  14991.088737   
119  15362.896982  46.286432  26.263412  15341.063987  15377.037630   
137  15368.953404  41.683144  23.392710  15350.640430  15370.864323   
174  15571.007906  45.281664  28.206654  15547.232787  15584.547005   
175  15559.566838  39.058077  29.353611  15532.352833  15577.630013   
178  15541.088208  39.959528  29.809098  15510.726210  15567.569987   
179  15530.049798  35.243747  29.058886  15496.265878  15561.817643   
212  15491.589131  56.533005  31.222037  15450.927120  15480.549479   
213  15498.618557  60.526337  29.264676  15448.279287  15482.258984   
234  15464.075882  54.027221  

In [21]:
import numpy as np
def pointpos(x):
    if x['TotalSignal']==2:
        return x['Low']-1e-3
    elif x['TotalSignal']==1:
        return x['High']+1e-3
    else:
        return np.nan

df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)

/var/folders/b9/tv8kqzz13cq_hsmncf9m9bs00000gp/T/ipykernel_44888/2345118794.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)


In [23]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
st=100
dfpl = df[st:st+350]
#dfpl.reset_index(inplace=True)
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close']),

                go.Scatter(x=dfpl.index, y=dfpl['BBL_15_1.5'],
                           line=dict(color='green', width=1),
                           name="BBL"),
                go.Scatter(x=dfpl.index, y=dfpl['BBU_15_1.5'],
                           line=dict(color='green', width=1),
                           name="BBU"),
                go.Scatter(x=dfpl.index, y=dfpl['EMA_fast'],
                           line=dict(color='black', width=1),
                           name="EMA_fast"),
                go.Scatter(x=dfpl.index, y=dfpl['EMA_slow'],
                           line=dict(color='blue', width=1),
                           name="EMA_slow")])

fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="entry")

fig.show()

In [24]:
def SIGNAL():
    return df.TotalSignal

In [25]:
from backtesting import Strategy
from backtesting import Backtest

class MyStrat(Strategy):
    mysize = 5
    slcoef = 1.1
    TPSLRatio = 1.5
    rsi_length = 16

    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)
        #df['RSI']=ta.rsi(df.Close, length=self.rsi_length)

    def next(self):
        super().next()
        slatr = self.slcoef*self.data.ATR[-1]
        TPSLRatio = self.TPSLRatio

        # if len(self.trades)>0:
        #     if self.trades[-1].is_long and self.data.RSI[-1]>=90:
        #         self.trades[-1].close()
        #     elif self.trades[-1].is_short and self.data.RSI[-1]<=10:
        #         self.trades[-1].close()

        if self.signal1==2 and len(self.trades)==0:
            sl1 = self.data.Close[-1] - slatr
            tp1 = self.data.Close[-1] + slatr*TPSLRatio
            self.buy(sl=sl1, tp=tp1, size=self.mysize)

        elif self.signal1==1 and len(self.trades)==0:
            sl1 = self.data.Close[-1] + slatr
            tp1 = self.data.Close[-1] - slatr*TPSLRatio
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

# df.index = pd.to_datetime(df.index)
print(df)
bt = Backtest(df, MyStrat, cash=1000000, margin=1/30)

                      Datetime          Open          High           Low  \
0    2024-01-16 14:30:00+00:00  14905.665039  14938.710938  14881.009766   
1    2024-01-16 14:45:00+00:00  14876.389648  14910.701172  14866.360352   
2    2024-01-16 15:00:00+00:00  14902.736328  14934.326172  14892.898438   
3    2024-01-16 15:15:00+00:00  14934.433594  14977.805664  14925.548828   
4    2024-01-16 15:30:00+00:00  14974.114258  14992.057617  14954.064453   
...                        ...           ...           ...           ...   
1008 2024-03-11 18:30:00+00:00  16054.590820  16057.852539  16034.976562   
1009 2024-03-11 18:45:00+00:00  16057.845703  16076.823242  16053.979492   
1010 2024-03-11 19:00:00+00:00  16057.684570  16058.118164  16030.704102   
1011 2024-03-11 19:15:00+00:00  16030.795898  16034.460938  16012.697266   
1012 2024-03-11 19:30:00+00:00  16017.302734  16034.933594  16007.973633   

             Close     Volume  Dividends  Stock Splits      EMA_slow  \
0     14885.206

/var/folders/b9/tv8kqzz13cq_hsmncf9m9bs00000gp/T/ipykernel_44888/1193700521.py:38: UserWarning:

Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.



In [26]:
bt.run()

Start                                     0.0
End                                    1012.0
Duration                               1012.0
Exposure Time [%]                   17.966436
Equity Final [$]                997338.569093
Equity Peak [$]                 1000933.44936
Return [%]                          -0.266143
Buy & Hold Return [%]                 7.71145
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                   -0.465442
Avg. Drawdown [%]                   -0.075841
Max. Drawdown Duration                  615.0
Avg. Drawdown Duration                 100.75
# Trades                                 40.0
Win Rate [%]                             40.0
Best Trade [%]                       0.533428
Worst Trade [%]                     -2.299069
Avg. Trade [%]                    

In [14]:
bt.plot()

f:\Python\Lib\site-packages\backtesting\_plotting.py:659: UserWarning:

found multiple competing values for 'toolbar.active_drag' property; using the latest value

f:\Python\Lib\site-packages\backtesting\_plotting.py:659: UserWarning:

found multiple competing values for 'toolbar.active_scroll' property; using the latest value



GridPlot(id='p1966', ...)